In [25]:
import glob
import pandas as pd
import numpy as np
import re
import json
import pickle
from matplotlib import pylab as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [3]:
CURRENT_FOLDER = f'analysis/chime/'

In [4]:
chime_mfcc_filename = f'{CURRENT_FOLDER}chime_mfcc.csv'
raw_df = pd.read_csv(chime_mfcc_filename)

In [28]:
tvt_filename = f'{CURRENT_FOLDER}train_val_test.pkl'
with open(tvt_filename, 'rb') as readfile:
    train_val_test = pickle.load(readfile)

In [29]:
pca = PCA()
scaler = StandardScaler()
pca_90 = PCA(0.9)
pca_50 = PCA(0.5)
xtrain = scaler.fit_transform(train_val_test['X_train'])
pca.fit_transform(xtrain)
pca_50.fit_transform(xtrain)
pca_90.fit_transform(xtrain)

array([[-34.44255228,  -7.85984383, -14.26408728, ...,  -2.19877924,
         -1.05359889,  -0.79442866],
       [ 42.67418778,   1.22087045,   6.64160394, ...,   2.22967752,
         -0.687279  ,  -3.69279056],
       [-14.63477139,   7.11055933,   5.78524565, ...,  -0.44239342,
         -0.55647615,   1.29830201],
       ...,
       [  2.33764124,  10.46461175,  -8.31714126, ...,   2.90918809,
         -0.97938499,   0.4724944 ],
       [  0.24940182,  21.07588185,   6.53337924, ...,   0.74185434,
         -0.05362823,   0.66528404],
       [  2.24185271,  -3.67147669,  -5.24241008, ...,  -1.88942063,
         -1.93428452,  -0.51426086]])

In [32]:
print('pca', pca.n_components_)
print('pca', pca_50.n_components_)
print('pca', pca_90.n_components_)

pca 1244
pca 74
pca 318


In [35]:
test = [{'id': 1, 'v': 5}, {'id': 2, 'v': 2}, {'id': 3, 'v': 3}]

In [37]:
sorted(test, key=lambda obj: obj['v'], reverse=True)

[{'id': 1, 'v': 5}, {'id': 3, 'v': 3}, {'id': 2, 'v': 2}]